In [10]:
from bs4 import BeautifulSoup



In [35]:
path = './10.lif'
def get_bbox(path):
    with open(path, 'r') as f:
        soup = BeautifulSoup(f, 'lxml')
        box = soup.find('bndbox')
        keys = ['xmin', 'xmax', 'ymin', 'ymax']
        return {key:int(box.find(key).contents[0]) for key in keys}
get_bbox(path)

{'xmax': 200, 'xmin': 98, 'ymax': 147, 'ymin': 114}